In [ ]:
!pip install keras-swa==0.1.5

     |████████████████████████████████| 81kB 4.8MB/s 
  Created wheel for keras-swa: filename=keras_swa-0.1.5-cp37-none-any.whl size=9714 sha256=24c3c3a6298e9e2e59c848f0186576b191b593c1c30bac05084b111d6c2cb511
  Stored in directory: /root/.cache/pip/wheels/30/de/a7/f2a71861bb678729a45e8995dc2496118e875e64b89379c530
Successfully built keras-swa


In [ ]:
from swa.tfkeras import SWA
import tensorflow.keras.backend as K

In [ ]:
from keras.datasets import cifar10

In [ ]:
(trainX, trainy), (testX, testy) = cifar10.load_data()
trainX = trainX/255
testX = testX/255

170500096/170498071 [==============================] - 4s 0us/step


In [ ]:
import numpy as np
import random
import cv2
import os
from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dropout
from tensorflow.keras import backend as K

In [ ]:
lb = LabelBinarizer()
trainy = lb.fit_transform(trainy)
testy = lb.fit_transform(testy)

In [ ]:
def label_dict(img, labels):
  lab_lis = {}
  for i in range(10):
    lab_lis[i] = []
  data = list(zip(img, labels))
  for im, lab in data:
    idx = np.argmax(lab)
    lab_lis[idx].append((im, lab))
  return lab_lis

In [ ]:
t = label_dict(trainX, trainy)

In [ ]:
unlab_data = []
unlab_img_data = []
for i in range(len(t)):
  unlab_data.extend(t[i][4000:])
for x,y in unlab_data:
  unlab_img_data.append(x)

In [ ]:
unlab_img_data[0]

array([[[0.99607843, 0.99607843, 0.99607843],
        [0.98823529, 0.98823529, 0.98823529],
        [0.98823529, 0.98823529, 0.98431373],
        ...,
        [0.98823529, 0.98823529, 0.99215686],
        [0.98823529, 0.98823529, 0.98039216],
        [1.        , 1.        , 0.99607843]],

       [[1.        , 1.        , 1.        ],
        [0.99607843, 0.99607843, 0.99607843],
        [0.99607843, 0.99607843, 0.98823529],
        ...,
        [0.99607843, 0.99607843, 1.        ],
        [0.99607843, 0.99607843, 0.98823529],
        [1.        , 1.        , 1.        ]],

       [[1.        , 1.        , 1.        ],
        [0.99607843, 0.99607843, 0.99607843],
        [1.        , 0.99607843, 0.99215686],
        ...,
        [0.99607843, 0.99607843, 1.        ],
        [0.99607843, 0.99607843, 0.98823529],
        [1.        , 1.        , 1.        ]],

       ...,

       [[0.96862745, 0.94509804, 0.95294118],
        [0.96078431, 0.93333333, 0.94117647],
        [0.96470588, 0

In [ ]:
client_names = ["client_"+str(i) for i in range(10)]
client_data = {}
for client in client_names:
  client_data[client] = []
i = 0
for client in client_names:
  client_data[client].extend(t[i%10][:1960])
  client_data[client].extend(t[(i+1)%10][1960:3920])
  i += 1
for i in range(10):
  k = 0
  for j in range(10):
    if j != (i)%10 and j != (i+1)%10:
      client_data["client_"+str(j)].extend(t[i][3920 + 10*k: 3920 + 10*(k+1)])
      k += 1

In [ ]:
def batch(zip_data, batch_size = 40):
  data, label = zip(*zip_data)
  dataset = tf.data.Dataset.from_tensor_slices((list(data), list(label)))
  return dataset.shuffle(len(label)).batch(batch_size)

In [ ]:
client_batched = {}
for (name, data) in client_data.items():
  client_batched[name] = batch(data)

test_batched = tf.data.Dataset.from_tensor_slices((testX, testy)).batch(len(testy))
unlab_dataset = tf.data.Dataset.from_tensor_slices((unlab_img_data)).batch(len(unlab_img_data))

In [ ]:
class MLP:
    @staticmethod
    def build():
        model = Sequential()
        model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
        model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        model.add(AveragePooling2D((2, 2)))
        model.add(Dropout(0.2))
        model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        model.add(AveragePooling2D((2, 2)))
        model.add(Dropout(0.2))
        model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        model.add(AveragePooling2D((2, 2)))
        model.add(Dropout(0.2))
        model.add(Flatten())
        model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
        model.add(Dropout(0.2))
        model.add(Dense(10, activation='softmax'))
        return model

In [ ]:
model = MLP()
model = model.build()

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
average_pooling2d (AveragePo (None, 16, 16, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)        36928     
_________________________________________________________________
average_pooling2d_1 (Average (None, 8, 8, 64)          0

In [ ]:
t = model.get_weights()
for v in t:
  print(np.shape(v))

(3, 3, 3, 32)
(32,)
(3, 3, 32, 32)
(32,)
(3, 3, 32, 64)
(64,)
(3, 3, 64, 64)
(64,)
(3, 3, 64, 128)
(128,)
(3, 3, 128, 128)
(128,)
(2048, 128)
(128,)
(128, 10)
(10,)


In [ ]:
comms_round = 50
loss='categorical_crossentropy'
metrics = ['accuracy']
optimizer = SGD(lr=0.01, 
                momentum=0.9
               )   
optimizer2 = SGD(lr=0.001,
                 momentum=0.9
               )   

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Tue May 11 18:33:21 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P0    31W /  70W |    230MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# model = MLP()
# model = model.build()
# model.compile(loss=loss, 
#                       optimizer=optimizer2, 
#                       metrics=metrics)
# print((model.optimizer.lr))
# model2 = MLP()
# model2 = model2.build()
# model2.compile(loss=loss, 
#                       optimizer=optimizer, 
#                       metrics=metrics)
# print((model2.optimizer.lr))

In [ ]:
weight_scale = []
for (name, data) in client_data.items():
  weight_scale.append(len(data)/40000)
weight_scale = np.array(weight_scale).reshape((len(weight_scale), 1))

In [ ]:
# def scale_model_weights(weight, scalar):
#     weight_final = []
#     steps = len(weight)
#     for i in range(steps):
#         weight_final.append(scalar * weight[i])
#     return weight_final

def sum_scaled_weights(scaled_weight_list, M, unlab_data):
    #print(type(scaled_weight_list[0][0]))
    temp = MLP()
    model = temp.build()
    p = np.zeros((10000,10))
    for i in range(len(weight_scale)):
      model.set_weights(scaled_weight_list[i])
      p += model.predict(unlab_data)
    avg_grad_mean = list()
    avg_grad_var = list()
    for grad_list_tuple in zip(*scaled_weight_list):
        #print(np.shape(grad_list_tuple))
        if len(np.shape(grad_list_tuple)) == 5:
          w = weight_scale.reshape((len(weight_scale), 1, 1, 1, 1))
        elif len(np.shape(grad_list_tuple)) == 4:
          w = weight_scale.reshape((len(weight_scale), 1, 1, 1))
        elif len(np.shape(grad_list_tuple)) == 3:
          w = weight_scale.reshape((len(weight_scale), 1, 1))
        else:
          w = weight_scale
        layer_mean = np.sum(grad_list_tuple*w, axis=0)
        #print(np.shape(layer_mean))
        layer_variance = np.sum(np.power(grad_list_tuple - layer_mean, 2)*w, axis = 0) 
        avg_grad_mean.append(layer_mean)
        avg_grad_var.append(layer_variance)
    model.set_weights(avg_grad_mean)
    p += model.predict(unlab_data)
    for i in range(M):
      grad_list = list()
      for j in range(len(avg_grad_var)):
        grad_list.append(np.random.normal(avg_grad_mean[j], avg_grad_var[j]))
      model.set_weights(grad_list)
      p += model.predict(unlab_data)
    return p/(M + len(weight_scale) + 1), avg_grad_mean


def test_model(X_test, Y_test,  model, comm_round):
    cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    logits = model.predict(X_test)
    loss = cce(Y_test, logits)
    acc = accuracy_score(tf.argmax(logits, axis=1), tf.argmax(Y_test, axis=1))
    print('comm_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, acc, loss))
    return acc, loss

In [ ]:
mlp_global = MLP()
global_model = mlp_global.build()

for comm_round in range(comms_round):
            
    
    global_weights = global_model.get_weights()
    
    
    scaled_local_weight_list = list()
    client_names = list(client_batched.keys())
    random.shuffle(client_names)
    
    for client in client_names:
        def scheduler(epoch, lr):
            if epoch  == 8:
              lr = lr*0.1
            if epoch == 15:
              lr = lr*0.1  
            return lr
        mlp_local = MLP()
        local_model = mlp_local.build()
        local_model.compile(loss=loss, 
                      optimizer=SGD(lr=0.01,momentum=0.9), 
                      metrics=metrics)
        local_model.set_weights(global_weights)
        
        callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
        local_model.fit(client_batched[client], epochs=25, verbose=0, callbacks=[callback])
        scaled_local_weight_list.append(local_model.get_weights())
        
        K.clear_session()
    p, avg = sum_scaled_weights(scaled_local_weight_list, 10, unlab_dataset)
    p = p**2/(np.sum(p**2, axis = 1).reshape((len(p), 1)))

    lab_dataset = tf.data.Dataset.from_tensor_slices((unlab_img_data,p)).batch(128)
    mlp_global = MLP()
    global_model = mlp_global.build()
    global_model.compile(loss=loss, 
                      optimizer=SGD(lr=0.001,momentum=0.9), 
                      metrics=metrics)
    global_model.set_weights(avg)
    swa = SWA(start_epoch=5, 
          lr_schedule='cyclic', 
          swa_lr=0.0004,
          swa_lr2=0.001,
          swa_freq=5,
          verbose=0)
    global_model.fit(lab_dataset, epochs = 20, verbose = 0, callbacks=[swa])

    for(X_test, Y_test) in test_batched:
        global_acc, global_loss = test_model(X_test, Y_test, global_model, comm_round)

comm_round: 0 | global_acc: 16.640% | global_loss: 2.28967547416687
comm_round: 1 | global_acc: 34.950% | global_loss: 2.213949203491211
comm_round: 2 | global_acc: 44.900% | global_loss: 2.1034395694732666
comm_round: 3 | global_acc: 51.120% | global_loss: 2.027338743209839
comm_round: 4 | global_acc: 54.360% | global_loss: 1.9775562286376953
comm_round: 5 | global_acc: 57.390% | global_loss: 1.939963698387146
comm_round: 6 | global_acc: 59.610% | global_loss: 1.91257905960083
comm_round: 7 | global_acc: 61.170% | global_loss: 1.8934775590896606
comm_round: 8 | global_acc: 62.420% | global_loss: 1.8772881031036377
comm_round: 9 | global_acc: 63.480% | global_loss: 1.866194486618042
comm_round: 10 | global_acc: 64.620% | global_loss: 1.8543281555175781
comm_round: 11 | global_acc: 65.300% | global_loss: 1.8467133045196533
comm_round: 12 | global_acc: 66.110% | global_loss: 1.8391326665878296
comm_round: 13 | global_acc: 66.540% | global_loss: 1.8312523365020752
comm_round: 14 | global_